In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

traindata=pd.read_csv("data/train.csv")
traindata.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
traindata.drop(["PassengerId","Name","Ticket","Cabin"],inplace=True,axis=1)
traindata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [3]:
traindata = traindata.fillna(traindata.mean())
traindata.isnull().sum()

C:\Users\Neha\AppData\Local\Temp\ipykernel_12484\1517838287.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  traindata = traindata.fillna(traindata.mean())


Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    2
dtype: int64

In [14]:

traindata['Embarked'].mode()[0]

'S'

In [5]:
traindata['Embarked'] = traindata['Embarked'].fillna(traindata['Embarked'].mode()[0])

In [6]:
traindata.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [7]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
X_train,X_test,y_train,y_test=train_test_split(traindata.iloc[:,1::],traindata.iloc[:,0],test_size=0.2,random_state=42)

In [8]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(712, 7) (179, 7) (712,) (179,)


In [9]:
encoder=OneHotEncoder(drop="first")
scaler=StandardScaler()
encoded_Xtrain=encoder.fit_transform(X_train[["Sex","Embarked"]]).toarray()
finalXtrain=np.hstack((X_train[["Pclass","Age","SibSp","Parch","Fare"]].values,encoded_Xtrain))
encoded_Xtrain_scaled=scaler.fit_transform(finalXtrain)
encoded_Xtrain_scaled

array([[-1.61413602,  1.22920747, -0.47072241, ...,  0.7243102 ,
        -0.30335547,  0.59248936],
       [-0.40055118, -0.50350514, -0.47072241, ...,  0.7243102 ,
        -0.30335547,  0.59248936],
       [ 0.81303367,  0.18957991, -0.47072241, ...,  0.7243102 ,
        -0.30335547,  0.59248936],
       ...,
       [ 0.81303367,  0.88266495,  1.23056874, ...,  0.7243102 ,
        -0.30335547,  0.59248936],
       [-1.61413602, -1.19659018,  0.37992316, ..., -1.38062393,
        -0.30335547,  0.59248936],
       [-1.61413602, -0.65752403, -0.47072241, ...,  0.7243102 ,
        -0.30335547,  0.59248936]])

In [41]:
clf=SVC(**params)
clf.fit(encoded_Xtrain_scaled,y_train)

SVC(kernel='linear')

In [37]:
params

{'kernel': 'linear', 'C': 1.0}

In [35]:
encoded_Xtest=encoder.fit_transform(X_test[["Sex","Embarked"]]).toarray()
finalXtest=np.hstack((X_test[["Pclass","Age","SibSp","Parch","Fare"]].values,encoded_Xtest))
encoded_Xtest_scaled=scaler.fit_transform(finalXtest)

In [42]:
ypred=clf.predict(encoded_Xtest_scaled)
accuracy_score(y_test,ypred)

0.7821229050279329

In [43]:
def data_transformation(data):

     # one hot encoding + standardization
    encoder=OneHotEncoder(drop="first")
    scaler=StandardScaler()
    encoded_data=encoder.fit_transform(data[["Sex","Embarked"]]).toarray()
    finaldata=np.hstack((data[["Pclass","Age","SibSp","Parch","Fare"]].values,encoded_data))
    encoded_scaled_data=scaler.fit_transform(finaldata)
    return encoded_scaled_data
    
def split_data(data):
    """Split a dataframe into training and validation datasets"""
    # fill missing values
    data.drop(["PassengerId","Name","Ticket","Cabin"],inplace=True,axis=1)
    data = data.fillna(data.mean())
    data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])
    features = data.drop(['Survived'], axis=1)
    labels = np.array(data['Survived'])
    
    features_train, features_valid, labels_train, labels_valid = \
        train_test_split(features, labels, test_size=0.2,
                         random_state=0)
    # data scaling
    features_train_scaled = data_transformation(features_train)
    features_valid_scaled = data_transformation(features_valid)

    return (features_train_scaled, labels_train,features_valid_scaled, labels_valid)

#kernels = [‘linear’, ‘rbf’, ‘poly’] C=1
# Train the model, return the model
def train_model(data, parameters):
    """Train a model with the given datasets and parameters"""
    # The object returned by split_data is a tuple.
    # Access train_data with data[0] and valid_data with data[1]

    train_data = data[0]
    labels_train = data[1]
    
    clf=SVC(**parameters)
    clf.fit(train_data,labels_train)
    return clf

# Evaluate the metrics for the model
def get_model_metrics(model, data):
    """Construct a dictionary of metrics for the model"""
    predictions = model.predict(data[2])
    fpr, tpr, thresholds = metrics.roc_curve(data[3], predictions)
    Acc = accuracy_score(data[3], predictions)
    model_metrics = {
        "auc": (
            metrics.auc(
                fpr, tpr)),
        "accuracy_score" : Acc
    
    }
    print(model_metrics)
    return model_metrics

In [23]:
params = {
    'kernel': 'linear',
    'C': 1.0
}

In [20]:
#!pip install lightgbm

In [21]:
df = pd.read_csv("data/train.csv")
train_df= split_data(df)
train_df

C:\Users\Neha\AppData\Local\Temp\ipykernel_12484\470182468.py:15: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data = data.fillna(data.mean())


(array([[ 8.19250590e-01, -2.82437263e-03, -4.64452339e-01, ...,
         -1.37207547e+00, -3.14269681e-01, -1.63985340e+00],
        [-3.80968381e-01,  9.66293694e-02, -4.64452339e-01, ...,
          7.28822884e-01, -3.14269681e-01,  6.09810609e-01],
        [-3.80968381e-01,  9.66293694e-02,  4.12709638e-01, ...,
          7.28822884e-01, -3.14269681e-01, -1.63985340e+00],
        ...,
        [ 8.19250590e-01, -2.82437263e-03, -4.64452339e-01, ...,
          7.28822884e-01,  3.18198052e+00, -1.63985340e+00],
        [ 8.19250590e-01,  4.78884313e-01,  4.12709638e-01, ...,
         -1.37207547e+00, -3.14269681e-01,  6.09810609e-01],
        [-3.80968381e-01,  2.31370804e+00,  4.12709638e-01, ...,
          7.28822884e-01, -3.14269681e-01,  6.09810609e-01]]),
 array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1,
        0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
        0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1,
       

In [44]:
model = train_model(train_df, params)

In [47]:
metrics = get_model_metrics(model, train_df)
metrics.items()

{'auc': 0.7732542819499342, 'accuracy_score': 0.7877094972067039}


dict_items([('auc', 0.7732542819499342), ('accuracy_score', 0.7877094972067039)])